In [4]:
# Importing required packages
import pandas as pd
import datetime

# Changing the pandas options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# To filter the warnings in the code
import warnings
warnings.filterwarnings("ignore")


In [8]:
# Reading the excel file
xls = pd.ExcelFile('Vendor Payment Data Set.xlsx')

In [9]:
# Creating seperate dataframes for each sheet
df1 = pd.read_excel(xls, "Invoice  Details ")
df2 = pd.read_excel(xls, "Payment Term table")
df3 = pd.read_excel(xls, "Payment Calender ")

# Step 1  Filter required columns for each dataset
## Invoice Details

    - 'Company Code', 'Vendor Account Number', 'Clearing Date', 'Posting Date in the Document', 'Document Date in Document', 'Day On Which Accounting Document Was Entered', 'Posting Key', 'Terms of Payment Key', 'Payment Frequency'
    
## Payment Calendar
    - ZZ_DIA, vendor, companyId
    
## Payment Term tables
    - 'zterm','ztagg', 'zfael', 'zmona', 'ztag1', 'ztag2', 'ztag3', 'zstg1', 'zstg2', 'zstg3','zsmn1', 'zsmn2', 'zsmn3','zdart'

In [11]:
invoice = df1[['Company Code', 'Vendor Account Number', 'Clearing Date', 'Posting Date in the Document', 'Document Date in Document', 'Day On Which Accounting Document Was Entered', 'Posting Key', 'Terms of Payment Key', 'Payment Frequency']]
terms = df2[['zterm','ztagg', 'zfael', 'zmona', 'ztag1', 'ztag2', 'ztag3', 'zstg1', 'zstg2', 'zstg3','zsmn1', 'zsmn2', 'zsmn3','zdart']]
calendar = df3[['Company Code ', 'Vendor Number', 'ZZ_DIA']]


# Step 2 - Cleaning the Data

## Step 2.1
- remove the rows with null values in the dataframes

In [12]:
invoice.isna().sum()

Company Code                                        2
Vendor Account Number                               3
Clearing Date                                       3
Posting Date in the Document                        3
Document Date in Document                           3
Day On Which Accounting Document Was Entered        3
Posting Key                                         4
Terms of Payment Key                            20161
Payment Frequency                                   7
dtype: int64

### Observation

* From the above cells we can see that some of the invoices contain null values
* We cannot calculate if the invoice is on time if they contain null values in company code, vendor number or terms of payment key.
* Terms of payment key is needed to find the terms of the record

**Because of that I'm dropping the invoices which contain any null values**

In [13]:
invoice = invoice.dropna()

## Step 2.2
- Changing the datatypes of columns to appropriate ones
    - Ex: Changing the datatype of Clearing Date from default str to Datetime object

In [14]:
invoice['Document Date in Document'] = invoice['Document Date in Document'].astype('datetime64[ns]')
invoice['Day On Which Accounting Document Was Entered'] = invoice['Day On Which Accounting Document Was Entered'].astype('datetime64[ns]')
invoice['Clearing Date'] = invoice['Clearing Date'].astype('datetime64[ns]')


## Step 2.3
- Cleaning the column names as some of the column names contain extra spaces at the end

In [15]:
invoice.columns = [column.strip() for column in invoice.columns]
terms.columns = [column.strip() for column in terms.columns]
calendar.columns = [column.strip() for column in calendar.columns]

# Step 3 - Merge the tables 

## Step 3.1 - Invoice and Terms:
- Merge the Invoice and Terms tables to identify the terms for each invoice
- Merge on Terms of Payment key in Invoice and zterm on terms 
 

In [16]:
final = invoice.merge(terms, left_on='Terms of Payment Key', right_on='zterm', how='left')

In [17]:
final.head()

,Company Code,Vendor Account Number,Clearing Date,Posting Date in the Document,Document Date in Document,Day On Which Accounting Document Was Entered,Posting Key,Terms of Payment Key,Payment Frequency,zterm,ztagg,zfael,zmona,ztag1,ztag2,ztag3,zstg1,zstg2,zstg3,zsmn1,zsmn2,zsmn3,zdart
0,BR11,390268,2022-01-04,2021-10-21,2021-10-20,2021-10-21,31.0,I075,Bi-Monthly,I075,0,0,0,75,0,0,0,0,0,0,0,0,B
1,BR11,312478,2022-01-04,2021-10-20,2021-10-19,2021-10-20,31.0,I065,Bi-Monthly,I065,0,0,0,65,0,0,0,0,0,0,0,0,B
2,BR11,406052,2022-01-04,2021-09-22,2021-09-22,2021-09-22,31.0,I095,Bi-Monthly,I095,0,0,0,95,0,0,0,0,0,0,0,0,B
3,BR11,405179,2022-01-04,2021-10-23,2021-10-22,2021-10-23,31.0,I065,Bi-Monthly,I065,0,0,0,65,0,0,0,0,0,0,0,0,B
4,BR11,390268,2022-01-04,2021-10-10,2021-10-09,2021-10-10,31.0,I075,Bi-Monthly,I075,0,0,0,75,0,0,0,0,0,0,0,0,B


## Step 3.2 - Merge Final and Calendar tables:
- Merge the ZZ_DIA column for each Company Code and Vendor Number  
    - Ex: 
        - If for a company with Company Code = 1 and Vendor Number = 10 there are two ZZ_DIA = 1, 14
        - Then merge the ZZ_DIA into one list = {1,14}
        - This way it is easier to merge with the Invoice table
- Merge the Invoice and New_Calendar tables to identify the ZZ_DIA for each invoice

In [18]:
new_calendar = calendar.groupby(['Company Code', 'Vendor Number'])['ZZ_DIA'].apply(list).to_frame("New DIA").reset_index()


In [19]:
# Top 10 records in the calendar are as follows
new_calendar.head(10)

,Company Code,Vendor Number,New DIA
0,BO11,384634,[1]
1,BR11,300049,[1]
2,BR11,300066,[1]
3,BR11,300071,[1]
4,BR11,300088,[1]
5,BR11,300110,[1]
6,BR11,300131,[1]
7,BR11,300160,[1]
8,BR11,300161,[1]
9,BR11,300170,[1]


### Observation

- From the NEW DIA column we can observe that it is dependent on Payment Frequency	
* Ex 1: If the Payment Frequency for a company and vendor pair is Monthly then the NEW DIA contains just one value {1}
* Ex 2: If the Payment Frequency for a company and vendor pair is Bi-Weekly then the NEW DIA contains two values {1,14} and so on

In [20]:
final = final.merge(new_calendar, left_on=['Company Code', 'Vendor Account Number'], right_on=['Company Code', 'Vendor Number'], how='left')


In [21]:
final.head()

,Company Code,Vendor Account Number,Clearing Date,Posting Date in the Document,Document Date in Document,Day On Which Accounting Document Was Entered,Posting Key,Terms of Payment Key,Payment Frequency,zterm,ztagg,zfael,zmona,ztag1,ztag2,ztag3,zstg1,zstg2,zstg3,zsmn1,zsmn2,zsmn3,zdart,Vendor Number,New DIA
0,BR11,390268,2022-01-04,2021-10-21,2021-10-20,2021-10-21,31.0,I075,Bi-Monthly,I075,0,0,0,75,0,0,0,0,0,0,0,0,B,390268.0,"[4, 14]"
1,BR11,312478,2022-01-04,2021-10-20,2021-10-19,2021-10-20,31.0,I065,Bi-Monthly,I065,0,0,0,65,0,0,0,0,0,0,0,0,B,312478.0,"[4, 14]"
2,BR11,406052,2022-01-04,2021-09-22,2021-09-22,2021-09-22,31.0,I095,Bi-Monthly,I095,0,0,0,95,0,0,0,0,0,0,0,0,B,406052.0,"[4, 14]"
3,BR11,405179,2022-01-04,2021-10-23,2021-10-22,2021-10-23,31.0,I065,Bi-Monthly,I065,0,0,0,65,0,0,0,0,0,0,0,0,B,405179.0,"[4, 14]"
4,BR11,390268,2022-01-04,2021-10-10,2021-10-09,2021-10-10,31.0,I075,Bi-Monthly,I075,0,0,0,75,0,0,0,0,0,0,0,0,B,390268.0,"[4, 14]"


In [22]:
final.isna().sum()

Company Code                                        0
Vendor Account Number                               0
Clearing Date                                       0
Posting Date in the Document                        0
Document Date in Document                           0
Day On Which Accounting Document Was Entered        0
Posting Key                                         0
Terms of Payment Key                                0
Payment Frequency                                   0
zterm                                               0
ztagg                                               0
zfael                                               0
zmona                                               0
ztag1                                               0
ztag2                                               0
ztag3                                               0
zstg1                                               0
zstg2                                               0
zstg3                       

### Observation

* Looks like some of the invoices do not contain ZZ_DIA values and also zdart values
* Dropping these invoices because we cannot calculate if the invoice had cleared on time.

In [23]:
final = final.dropna()

### Final Data after cleaning

In [24]:
final.describe(include='object')

,Company Code,Vendor Account Number,Terms of Payment Key,Payment Frequency,zterm,zdart,New DIA
count,280392,280392,280392,280392,280392,280392,280392
unique,7,7119,126,4,126,1,14
top,BR16,318014,I065,Bi-Monthly,I065,B,"[4, 14]"
freq,229799,10928,70753,142321,70753,280392,138082


In [25]:
final.describe()

,Posting Key,ztagg,zfael,zmona,ztag1,ztag2,ztag3,zstg1,zstg2,zstg3,zsmn1,zsmn2,zsmn3,Vendor Number
count,280392.0,280392.0,280392.0,280392.0,280392.000000,280392.0,280392.0,280392.0,280392.0,280392.0,280392.0,280392.0,280392.0,280392.000000
mean,31.0,0.0,0.0,0.0,85.812702,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,357489.014936
std,0.0,0.0,0.0,0.0,40.303126,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44189.675701
min,31.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,300049.000000
25%,31.0,0.0,0.0,0.0,65.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,315658.000000
50%,31.0,0.0,0.0,0.0,75.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,365680.000000
75%,31.0,0.0,0.0,0.0,105.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,400194.000000
max,31.0,0.0,0.0,0.0,296.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,436893.000000


# Step 4: Define a function to identify on time invoices 

## Assumptions

* Assuming that invoices with any null values can not be identified if they are on time
* After cleaning the data I can see that zdart contains only one unique value 'B' 
* And also all the term values are zero except for 'ztag1'
* Assuming that only those are present for calculation

* Assuming that for a monthly payment frequency if the net due date is after 1st of month then the expected date would be 1st of next month

In [26]:
def expected_date(row):
    if row['zdart']=='B':
        maximum_allowable_date = row['Document Date in Document'] 
    elif row['zdart'] == 'C':
        maximum_allowable_date = row['Day On Which Accounting Document Was Entered'] 
    elif row['zdart'] == 'D':
        maximum_allowable_date = row['Posting Date in the Document']
    

    maximum_allowable_date += datetime.timedelta(days=row['ztag1'])
    # Finding the correct day from the ZZ_DIA values
    day = maximum_allowable_date.day
    month = maximum_allowable_date.month
    year = maximum_allowable_date.year
    flag = 0
    for max_day in row['New DIA']:
        if max_day >= day:
            flag = 1
            break
    
    if not flag:
        maximum_allowable_date = datetime.datetime(year + int(month / 12), ((month % 12) + 1), row['New DIA'][0])
    
    else:
        maximum_allowable_date = datetime.datetime(year, month, max_day)
    return maximum_allowable_date

In [27]:
final['Expected Date'] = final.apply(expected_date, axis=1)

In [28]:
final.head()

,Company Code,Vendor Account Number,Clearing Date,Posting Date in the Document,Document Date in Document,Day On Which Accounting Document Was Entered,Posting Key,Terms of Payment Key,Payment Frequency,zterm,ztagg,zfael,zmona,ztag1,ztag2,ztag3,zstg1,zstg2,zstg3,zsmn1,zsmn2,zsmn3,zdart,Vendor Number,New DIA,Expected Date
0,BR11,390268,2022-01-04,2021-10-21,2021-10-20,2021-10-21,31.0,I075,Bi-Monthly,I075,0,0,0,75,0,0,0,0,0,0,0,0,B,390268.0,"[4, 14]",2022-01-04
1,BR11,312478,2022-01-04,2021-10-20,2021-10-19,2021-10-20,31.0,I065,Bi-Monthly,I065,0,0,0,65,0,0,0,0,0,0,0,0,B,312478.0,"[4, 14]",2022-01-04
2,BR11,406052,2022-01-04,2021-09-22,2021-09-22,2021-09-22,31.0,I095,Bi-Monthly,I095,0,0,0,95,0,0,0,0,0,0,0,0,B,406052.0,"[4, 14]",2022-01-04
3,BR11,405179,2022-01-04,2021-10-23,2021-10-22,2021-10-23,31.0,I065,Bi-Monthly,I065,0,0,0,65,0,0,0,0,0,0,0,0,B,405179.0,"[4, 14]",2022-01-04
4,BR11,390268,2022-01-04,2021-10-10,2021-10-09,2021-10-10,31.0,I075,Bi-Monthly,I075,0,0,0,75,0,0,0,0,0,0,0,0,B,390268.0,"[4, 14]",2022-01-04


In [35]:
(final['Clearing Date'] <= final['Expected Date']).sum()/invoice.shape[0]

0.6711008054674151

In [34]:
final.shape

(280392, 26)